In [45]:
:dep darn
:dep ndarray
:dep ndarray-csv
:dep polars = { version = "x", features = ["csv","describe","fmt","lazy","ndarray","dtype-categorical","to_dummies"]}
:dep plotters = {version = "0.3.5", features = ["evcxr", "line_series"]}
:dep linfa
:dep linfa-bayes
:dep linfa-preprocessing
:sccache 1
    
extern crate csv;

use std::io::prelude::*;
use std::fs::*;
use std::path::Path
use ndarray::prelude::*;
use ndarray_csv::Array2Reader;
use ndarray::*;

use polars::prelude::*;
use std::fs::File;

use linfa::*;
use linfa::prelude::*;

use linfa::traits::Transformer;
use linfa_preprocessing::norm_scaling::NormScaler;
use linfa_preprocessing::linear_scaling::LinearScaler;
use linfa_bayes::{GaussianNbParams, GaussianNbValidParams, Result};

extern crate plotters;
use plotters::prelude::*;
use plotters::series::*;


sccache: true. Warning: dynamic linking disabled, use :cache instead to preserve dynamic linking


In [70]:
// import database
let df = CsvReader::from_path("./db/risco_credit.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

let df= df.clone().lazy();



let x = df.clone().select([col("*").exclude(["risco"])])
    .with_columns([
        col("historia").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("divida").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("garantias").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical(),
        col("renda").cast(DataType::Categorical(None,CategoricalOrdering::Physical)).to_physical()     
    ])
    .drop_nulls(None).collect();
println!("DataFrame {:?}",x);


let y = df.clone()
    .select([col("risco")])
    .drop_nulls(None)
    .collect();

let y = y?.column("risco")?.clone();


{

let ndarray_x = x?.clone().to_ndarray::<Float32Type>(IndexOrder::C).unwrap();

let ndarray_y:Vec<String> =   y.str()?.
                                    into_iter().
                                    filter_map(|opt_str| opt_str.map(|s| s.to_string()))
                                    .collect();
    
let ds = Dataset::new(ndarray_x, ndarray_y.into());
    
    {
        let (train, test) = ds.split_with_ratio(1.);
        
        let unchecked_params = GaussianNbParams::new()
        .var_smoothing(1e-5);

        let model = unchecked_params.fit(&train);
        
         let model_safe = match model {
            Ok(model) => model,
            Err(error) => {
                // panic, or you may also want to handle it in other ways
                panic!("Problem fitting the model: {:?}", error);
            }
        };

        
        //historia boa 0, divida alta 0. garantias nenhuma 1, renda > 35 2
        //historia ruim 2, divida alta 0, garantias adequada 0, remda < 15 0
        let test_values = array![[0.0,0.0,1.0,2.0],[2.0,0.0,0.0,0.0]];
        
        let predictions = model_safe.predict(&test_values);
        
        // let cm = predictions.confusion_matrix(&test).unwrap();

        // println!("Confusion Matrix ");
        println!("{:?}", predictions);
        // println!("{:?}", cm);


        
    }
}

DataFrame Ok(shape: (14, 4)
┌──────────┬────────┬───────────┬───────┐
│ historia ┆ divida ┆ garantias ┆ renda │
│ ---      ┆ ---    ┆ ---       ┆ ---   │
│ u32      ┆ u32    ┆ u32       ┆ u32   │
╞══════════╪════════╪═══════════╪═══════╡
│ 0        ┆ 0      ┆ 0         ┆ 0     │
│ 1        ┆ 0      ┆ 0         ┆ 1     │
│ 1        ┆ 1      ┆ 0         ┆ 1     │
│ 1        ┆ 1      ┆ 0         ┆ 2     │
│ 1        ┆ 1      ┆ 0         ┆ 2     │
│ …        ┆ …      ┆ …         ┆ …     │
│ 2        ┆ 0      ┆ 1         ┆ 2     │
│ 2        ┆ 0      ┆ 0         ┆ 0     │
│ 2        ┆ 0      ┆ 0         ┆ 1     │
│ 2        ┆ 0      ┆ 0         ┆ 2     │
│ 0        ┆ 0      ┆ 0         ┆ 1     │
└──────────┴────────┴───────────┴───────┘)
["baixo", "alto"], shape=[2], strides=[1], layout=CFcf (0xf), const ndim=1


()